In [59]:
#imports
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()


True

In [2]:
#gets data collected into a csv
def get_csv(region='NA1',game_mode='ARAM',patch='14.13'):
    #if using sqlalchemy
    # engine_name = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    os.makedirs('MatchData', exist_ok=True)
    csv_path = f'MatchData/{region}_{game_mode}_{patch}.csv'

    cursor = conn.cursor()
    patch = patch+'%'

    query_sql = """SELECT * 
    FROM match_data 
    WHERE region = %s 
    AND game_mode = %s 
    AND patch LIKE %s"""

    query = cursor.mogrify(query_sql,(region, game_mode,patch))
    query = query.decode('utf-8')

    try:
        if os.path.exists(csv_path):
            print("Csv found")
        else:
            with open(csv_path,'w') as f:
                cursor.copy_expert("COPY ({}) TO STDOUT WITH CSV HEADER".format(query),f)
            print("Copy to csv successful")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [60]:
#features extraction, encoding and data verification
def get_data(region='NA1',game_mode='ARAM',elo='ANY',version='14.13'):
    engine_name = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    engine = create_engine(engine_name)
    
    version = version+'%'
    elo = elo+'%'

    if(elo == 'ANY%'):
        query_sql = """SELECT * 
        FROM match_data 
        WHERE region = %s 
        AND game_mode = %s 
        AND version LIKE %s"""
        params = (region,game_mode,version)
        
    else:
        query_sql = """SELECT * 
        FROM match_data 
        WHERE region = %s 
        AND game_mode = %s 
        AND elo LIKE %s
        AND version LIKE %s"""
        params = (region, game_mode, elo, version)

    df = pd.read_sql_query(query_sql,con=engine,params=params)
    print(df.shape)
    return df

In [66]:
def main(region,game_mode,elo,version):
    #get_csv(region=region,game_mode=game_mode,patch=patch)
    df = get_data(region,game_mode,elo,version)

main('NA1','ARAM','ANY','14.13')

(5072, 17)
